# 6 УРОК ГРУППИРОВКА ДАННЫХ

## Задача 1. 
В лекции мы познакомились с оператором GROUP BY и составили несколько базовых запросов с группировкой. Давайте на всякий случай ещё раз уточним логику, которая стоит за этой операцией:

Сначала в таблице определяются строки, в которых в указанном в GROUP BY столбце есть одинаковые значения.
Далее по этим значениям записи объединяются в группы, причём в группе может быть даже одна запись.
После этого над элементами этих групп, как правило, проводятся какие-то операции с помощью агрегирующих функций: например, с помощью SUM() вычисляется сумма значений в каком-либо столбце в каждой группе:
SELECT column_1, SUM(column_2)
FROM table
GROUP BY column_1


4. Затем в результирующий запрос попадает по одной уникальной записи из столбца, по которому осуществлялась группировка. При этом напротив каждого такого значения отражается результат агрегации по соответствующей группе, если агрегация проводилась.

Здесь важно сделать несколько уточнений.

Во-первых, группировка всегда выполняется после фильтрации, т.е. сначала выполняются инструкции в WHERE и только потом данные группируются через GROUP BY.

Во-вторых, к образовавшимся в результате применения GROUP BY группам можно применять сразу несколько агрегирующих функций (в том числе к разным колонкам).

В-третьих, группировку можно делать сразу по новым полям, посчитанным в SELECT: При этом допускается использование в GROUP BY алиаса колонки, указанного в SELECT. Следующие два запроса дадут одинаковый результат:

SELECT DATE(column_1) AS date, SUM(column_2)
FROM table
GROUP BY DATE(column_1)


SELECT DATE(column_1) AS date, SUM(column_2)
FROM table
GROUP BY date


В-четвёртых, делать агрегацию после группировки необязательно. Если не указывать агрегирующую функцию, то запрос вернёт уникальные значения в столбце, т.е. тот же результат, что и оператор DISTINCT. Можете самостоятельно запустить следующий запрос в Redash и убедиться:

SELECT user_id
FROM user_actions
GROUP BY user_id

SELECT DISTINCT user_id
FROM user_actions


Кстати, вопрос о том, какими способами можно отобрать уникальные значения, опытные аналитики любят задавать на собеседованиях своим менее опытным коллегам.

И наконец, последнее важное уточнение: при использовании группировки колонки, указанные в SELECT, должны находиться и в GROUP BY, если они не используются в агрегационных функциях. Это обязательное условие, и если оно не будет выполнено, то база данных вернёт ошибку.

Следующий запрос работать не будет, так как в GROUP BY указаны не все неагрегированные колонки из блока SELECT: 

SELECT column_1, column_2, SUM(column_3)
FROM table
GROUP BY column_1


В то же время такой запрос сработает:

SELECT SUM(column_2)
FROM table
GROUP BY column_1


Обратите внимание, что в этом запросе в блоке SELECT нет колонки, указанной в GROUP BY, т.е. в обратную сторону правило не работает: если мы что-то указали в GROUP BY, то это не обязательно указывать в SELECT. Иными словами, выводить наименования групп не обязательно.

И ещё: вместо названий колонок в блоке GROUP BY можно использовать номер колонки, указанной в SELECT. Например, следующие два запроса эквивалентны:

SELECT column_1, column_2, SUM(column_3)
FROM table
GROUP BY column_1, column_2


SELECT column_1, column_2, SUM(column_3)
FROM table
GROUP BY 1, 2


При этом номера колонок из SELECT можно также использовать при сортировке в операторе ORDER BY. Можете сами поэкспериментировать с этим в следующих заданиях.

С теорией вроде бы разобрались, теперь приступим к практике. В прошлом уроке мы уже считали количество курьеров женского пола и использовали для этого фильтрацию. Группировка позволит нам провести расчёты сразу для двух полов.

Задание:

С помощью группировки посчитайте количество курьеров мужского и женского пола в таблице couriers. Новую колонку с числом курьером назовите couriers_count. Результат отсортируйте по этой колонке по возрастанию.

Поля в результирующей таблице: sex, couriers_count

##### ОТВЕТ:

In [ ]:
SELECT sex, 
    COUNT(DISTINCT(courier_id)) AS couriers_count
FROM couriers
GROUP BY 1
ORDER BY 2


## Задача 2.
Теперь давайте аналогичным образом посчитаем максимальный возраст пользователей мужского и женского пола. Только в этот раз выведем не полный возраст, а только количество полных лет. Для этого к результату вычислений можно применить уже знакомую нам функцию DATE_PART с аргументом 'year'.

Задание:

Посчитайте максимальный возраст пользователей мужского и женского пола в таблице users. Возраст измерьте количеством полных лет. Новую колонку с возрастом назовите max_age. Результат отсортируйте по новой колонке по возрастанию возраста.

Поля в результирующей таблице: sex, max_age

##### ОТВЕТ:

In [ ]:
SELECT sex, 
    DATE_PART('year', AGE(current_date, MIN(birth_date))) AS max_age
FROM users
GROUP BY 1
ORDER BY 2

## Задача 3.
Маркетологи снова обратились к нам с задачей: в этот раз они просят провести небольшой анализ нашей аудитории и посчитать, сколько клиентов определённого возраста пользуются нашим сервисом. Давайте поможем нашим коллегам!

Задание:

Разбейте пользователей из таблицы users на группы по возрасту (возраст измеряем количеством полных лет) и посчитайте число пользователей каждого возраста. Колонку с возрастом назовите age, а колонку с числом пользователей — users_count. Отсортируйте полученный результат по возрастанию возраста.

Поля в результирующей таблице: age, users_count

Пояснение:

Обратите внимание, что в данной задаче вам необходимо провести группировку по новому расчётному полю с возрастом.

В этой ситуации важно помнить, что колонки, указанные в SELECT, должны находиться и в GROUP BY (если они не используются в агрегационных функциях). При этом в GROUP BY допускается использование алиаса колонки, указанного в блоке SELECT, т.е. повторно производить вычисления в GROUP BY не обязательно. 



##### ОТВЕТ:

In [ ]:
SELECT 
    DATE_PART('year', AGE(current_date, birth_date)) AS age,
    COUNT(user_id) as users_count
FROM users
GROUP BY 1
ORDER BY 1

## Задача 4.
Вы могли заметить, что результат предыдущего запроса для одной из групп вернул пустое значение возраста. Мы снова столкнулись с NULL значениями — на этот раз в колонке birth_date. Давайте избавимся от них перед группировкой и заодно сделаем наш анализ ещё более детальным: добавим в группировку пол пользователей.

Задание:

Вновь разбейте пользователей из таблицы users на группы по возрасту (возраст измеряем количеством полных лет), только теперь добавьте в группировку пол пользователя. В результате в каждой возрастной группе должно появиться ещё по две подгруппы с полом. В каждой такой подгруппе посчитайте число пользователей. Все NULL значения в колонке birth_date заранее отфильтруйте с помощью WHERE. Колонку с возрастом назовите age, а колонку с числом пользователей — users_count, имя колонки с полом оставьте без изменений. Отсортируйте полученную таблицу сначала по колонке с возрастом по возрастанию, затем по колонке с полом — тоже по возрастанию.

Поля в результирующей таблице: age, sex, users_count

##### ОТВЕТ:

In [ ]:
SELECT 
    DATE_PART('year', AGE( birth_date)) AS age,
    sex,
    COUNT(user_id) as users_count
FROM users
WHERE birth_date IS NOT NULL
GROUP BY 1,2
ORDER BY 1,2

## Задача 5.
А теперь, используя наши знания о группировке, давайте посчитаем, сколько заказов было сделано и сколько отменено в каждом отдельном месяце. В этот раз для работы с датами будем использовать не DATE_PART, а новую функцию DATE_TRUNC.

Функция DATE_TRUNC используется для усечения дат и времени, т.е. она работает почти как округление ROUND, только для типов данных TIMESTAMP и INTERVAL.

Синтаксис у неё такой же, как и у DATE_PART:

SELECT DATE_TRUNC(part, column)


На месте part в кавычках указывается, до какой точности следует обрезать переданное значение времени:  'year', 'month', 'day', 'hour' и т.д.

Возвращаемое значение имеет тип TIMESTAMP или INTERVAL, а все «части» исходного значения, менее значимые, чем заданная «часть», приравниваются к нулю (или единице, если это номер дня или месяца):

SELECT DATE_TRUNC('month', TIMESTAMP '2022-01-12 08:55:30')

Результат:
01/01/22 00:00

SELECT DATE_TRUNC('day', TIMESTAMP '2022-01-12 08:55:30')

Результат:
12/01/22 00:00	

SELECT DATE_TRUNC('hour', TIMESTAMP '2022-01-12 08:55:30')

Результат:
12/01/22 08:00	



На заметку:

Подробнее с функцией DATE_TRUNC можно ознакомиться здесь.

Задание:

Используя функцию DATE_TRUNC, посчитайте, сколько заказов было сделано и сколько было отменено в каждом месяце. Расчёты проводите по таблице user_actions. Колонку с усечённой датой назовите month, колонку с количеством заказов — orders_count. Результат отсортируйте сначала по месяцам — по возрастанию, затем по типу действия — тоже по возрастанию.

Поля в результирующей таблице: month, action, orders_count

##### ОТВЕТ:

In [ ]:
SELECT DATE_TRUNC('month',time) AS month,
        action,
        COUNT(order_id) AS orders_count
   
FROM user_actions

GROUP BY 1,2
ORDER BY 1,2


## Задача 6.
На прошлом уроке мы научились работать с функцией array_length и даже посчитали с её помощью количество товаров в каждом заказе. Давайте для каждого размера заказа, который встречается в данных, посчитаем общее число заказов такого размера.

Задание:

Посчитайте количество товаров в каждом заказе из таблицы orders, примените к этим значениям группировку и посчитайте количество заказов в каждой группе. Выведите две колонки: количество товаров в заказе и число заказов с таким количеством. Колонки назовите соответственно order_size и orders_count. Результат отсортируйте по возрастанию числа товаров в заказе.

Поля в результирующей таблице: order_size, orders_count

##### ОТВЕТ:

In [ ]:
SELECT ARRAY_LENGTH(product_ids,1) order_size, 
    COUNT(order_id) AS orders_count
FROM orders

GROUP BY 1
ORDER BY 1

## Задача 7.
Теперь давайте добавим к предыдущему запросу оператор HAVING, который рассматривали на лекции, и выполним фильтрацию уже после группировки.

Но перед этим обсудим одну важную особенность работы с HAVING: в этом блоке нельзя указывать алиасы расчётных полей из блока SELECT. Дело в том, что в соответствии с порядком выполнения запроса оператор SELECT выполняется после оператора HAVING. Поэтому агрегацию необходимо также указывать и в HAVING.

Например, такой запрос не сработает:

SELECT column_1, SUM(column_2) AS new_column
FROM table
GROUP BY column_1
HAVING new_column = 10


А такой сработает:

SELECT column_1, SUM(column_2) AS new_column
FROM table
GROUP BY column_1
HAVING SUM(column_2) = 10


Задание:

Дополните предыдущий запрос оператором HAVING и отберите только те размеры заказов, общее число которых превышает 5000. Вновь выведите две колонки: количество товаров в заказе и число заказов с таким количеством. Колонки назовите соответственно order_size и orders_count. Результат отсортируйте по возрастанию числа товаров в заказе.

Поля в результирующей таблице: order_size, orders_count



##### ОТВЕТ:

In [ ]:
SELECT ARRAY_LENGTH(product_ids,1) order_size, 
    COUNT(order_id) AS orders_count
FROM orders

GROUP BY 1
HAVING COUNT(order_id) > 5000
ORDER BY 1

## Задача 8.
Перед тем как двигаться дальше, предлагаем вам решить ещё пару задач на группировку.

Задание:

Из таблицы courier_actions отберите id трёх курьеров, доставивших в сентябре 2022 года наибольшее количество заказов. Выведите две колонки — id курьера и число доставленных заказов. Колонку с числом доставленных заказов назовите delivered_orders.

Поля в результирующей таблице: courier_id, delivered_orders

Пояснение:

Помните, что в таблице courier_actions есть информация как о созданных, так и о доставленных заказах.

##### ОТВЕТ:

In [ ]:
SELECT courier_id, 
        COUNT(order_id) AS delivered_orders
FROM courier_actions
WHERE action = 'deliver_order' AND (time > timestamp '2022-09-01')
GROUP BY courier_id
ORDER BY delivered_orders DESC
LIMIT 3

## Задача 9.
Задание:

А теперь отберите id только тех курьеров, которые в сентябре 2022 года успели доставить только по одному заказу. Таблица та же — courier_actions. Вновь выведите две колонки — id курьера и число доставленных заказов. Колонку с числом заказов назовите delivered_orders. Результат отсортируйте по возрастанию id курьера.

Поля в результирующей таблице: courier_id, delivered_orders

##### ОТВЕТ:

In [ ]:
SELECT courier_id, 
        COUNT(order_id) AS delivered_orders
FROM courier_actions
WHERE action = 'deliver_order' AND (time > timestamp '2022-09-01')
GROUP BY courier_id
HAVING COUNT(order_id) = 1
ORDER BY 1

## Задача 10.
Ой, к нам в кабинет снова постучались! Это опять маркетологи: говорят, что хотят разослать пуш-уведомление со специальным предложением. Аудитория — пользователи, которые давно не делали у нас заказ.

Задание:

Из таблицы user_actions отберите пользователей, у которых последний заказ был создан до 1 сентября 2022 года. Выведите только их id, дату создания заказа выводить не нужно. Результат отсортируйте по возрастанию id пользователя.

Поле в результирующей таблице: user_id

Если совсем не получается:

Чтобы решить задачу, нужно сначала для каждого пользователя найти последний timestamp, когда он совершал заказ, и затем отобрать тех, кто делал это слишком давно. Учитывайте, что в таблице есть не только время создания, но и время отмены заказа.

##### ОТВЕТ:

In [ ]:
SELECT user_id
FROM user_actions
WHERE action = 'create_order' 
GROUP BY 1
HAVING MAX(time) < timestamp '2022-09-01'
ORDER BY 1


## * Задача 11.
А теперь попробуем решить задачу посложнее. Для неё нам снова пригодится агрегатное выражение с фильтрацией, которое мы рассматривали на прошлом уроке. Эту конструкцию можно применять не только ко всей таблице, но и отдельно к каждой группе, сформированной в результате применения оператора GROUP BY. В общем виде она будет выглядеть так:

SELECT column_1, agg_function(column_2) FILTER (WHERE [condition])
GROUP BY column_1
FROM table


Пример:

SELECT column_1, AVG(column_2) FILTER (WHERE column_3 > 100)
GROUP BY column_1
FROM table


Задание:

Для каждого пользователя в таблице user_actions посчитайте долю отменённых заказов. Выведите две колонки: id пользователя и рассчитанный показатель. Новую колонку с показателем округлите до двух знаков после запятой и назовите cancel_rate. Результат отсортируйте по возрастанию id пользователя.

Поля в результирующей таблице: user_id, cancel_rate

Пояснение:

Чтобы посчитать долю отменённых заказов, необходимо поделить количество отменённых заказов на общее число уникальных заказов пользователя. Помните, что для получения корректного результата деления нужно хотя бы одно из значений привести к типу DECIMAL.



##### ОТВЕТ:

In [ ]:
select user_id, 
        round((count(order_id) 
        filter (where action='cancel_order'))/count(distinct order_id)::decimal,2) as cancel_rate
from user_actions
group by user_id
order by user_id

In [ ]:
SELECT 
    user_id,
    ROUND((COUNT(order_id)  FILTER (WHERE action = 'cancel_order')/ CAST(COUNT(DISTINCT(order_id)) AS DECIMAL) ), 2)   AS cancel_rate
FROM user_actions

GROUP BY 1
ORDER BY 1

## * Задача 12.
И в конце урока давайте вернёмся... Нет, не к налогам. Вернёмся мы к запросу с группировкой пользователей по возрасту, который делали на этом шаге.

Мы посчитали количество пользователей каждого возраста, но смотреть на данные именно в такой группировке не очень-то интересно. Давайте перейдём от конкретных значений возраста к возрастным группам.

Задание:

Разбейте пользователей из таблицы users на 4 возрастные группы:

от 19 до 24 лет;
от 25 до 29 лет;
от 30 до 35 лет;
от 36 до 41 года.
Посчитайте число пользователей, попавших в каждую возрастную группу. Группы назовите соответственно «19-24», «25-29», «30-35», «36-41» (без кавычек). Выведите наименования групп и число пользователей в них. Колонку с наименованием групп назовите group_age, а колонку с числом пользователей — users_count. Отсортируйте полученную таблицу по колонке с наименованием групп по возрастанию.

Поля в результирующей таблице: group_age, users_count

Пояснение:

Для решения этой задачи подойдёт конструкция CASE. Как и в прошлый раз, в качестве возраста рассматривайте количество полных лет.

Если совсем не получается:

Примерный алгоритм решения следующий: сначала с помощью CASE и логических выражений необходимо в новом расчётном столбце присвоить каждому пользователю группу, затем нужно сгруппировать данные по новому столбцу и провести подсчёт количества пользователей в каждой группе. Определение групп лучше проводить в SELECT, а в GROUP BY использовать алиас новой колонки. Не забудьте отфильтровать NULL значения.



##### ОТВЕТ:

In [ ]:
SELECT 
    CASE
        WHEN DATE_PART('year', AGE(birth_date)) BETWEEN 19 AND 24 THEN '19-24'
        WHEN DATE_PART('year', AGE(birth_date)) BETWEEN 25 AND 29 THEN '25-29'
        WHEN DATE_PART('year', AGE(birth_date)) BETWEEN 30 AND 35 THEN '30-35'
        WHEN DATE_PART('year', AGE(birth_date)) BETWEEN 36 AND 41 THEN '36-41'
    END AS group_age,
    COUNT(user_id) as users_count
FROM users
WHERE birth_date IS NOT NULL
GROUP BY 1
ORDER BY 1

## Подведём итоги
В этом уроке мы:

Научились группировать данные и узнали разные нюансы работы с оператором GROUP BY.
Поработали с агрегирующими функциями и научились применять их к сгруппированным данным.
Узнали, что к результату группировки можно применять фильтрацию с помощью оператора HAVING.
Ещё немного поработали с датами и познакомились с новой функцией DATE_TRUNC.
Опять коснулись продвинутой темы и научились применять агрегатные выражения поверх группировки.
Решили большую практическую задачу на CASE с группировкой.
Впереди нас ждут ещё более интересные темы и продвинутые запросы, где пригодятся все знания, которые мы получили на предыдущих уроках.